# Libraries

In [24]:
# Libraries
import os
import requests
import json
import folium
from pandas import json_normalize

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

# Show current working directory
print(os.getcwd())

/workspaces/project_arm_airbnb/data_maps


## API request

### Touristenattraktionen

In [27]:
# Overpass API URL
url = 'http://overpass-api.de/api/interpreter'

# Overpass turbo query
query = f"""
        [out:json];
        area["ISO3166-2"="CH-GE"][admin_level=4];
        node ["tourism"="attraction"](area);
        out;"""

# Web API request
r = requests.get(url, params={'data': query})
data = r.json()['elements']

# Save data to file
with open('attraction.json', 'w') as json_file:
    json.dump(data, json_file)

# Store data in data frame
df_attraction = json_normalize(data)

# Number of rows and columns
print(df_attraction.shape)

# First rows
df_attraction.head(50)

(37, 57)


,type,id,lat,lon,tags.addr:street,tags.amenity,tags.bottle,tags.dog,tags.drinking_water,tags.fountain,...,tags.internet_access,tags.leisure,tags.level,tags.opening_hours,tags.denotation,tags.geological,tags.length,tags.url,tags.width,tags.alt
0,node,60018172,46.207381,6.155889,Jetée des Eaux Vives,fountain,no,no,no,jet_water,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,node,139446272,46.128550,5.957381,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,node,151258480,46.132357,5.955911,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,node,613159167,46.142711,6.126112,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,node,916320563,46.224280,6.138078,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,node,983797939,46.232545,6.055259,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,node,1464505193,46.201098,6.147034,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,node,1895035255,46.200547,6.146384,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,node,2373193682,46.171219,6.135693,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,node,2481123835,46.229570,6.069080,NaN,drinking_water,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
# load data from df_attraction intro locations_attraction
locations_attraction = df_attraction[['lat','lon', "tags.name"]]

#Create map
m = folium.Map(location=[46.2044, 6.1432], zoom_start=12)

# add marker to sympols
for i in range(0, len(locations_attraction)):
    folium.Marker([locations_attraction.iloc[i]['lat'], locations_attraction.iloc[i]['lon']],
                  popup=locations_attraction.iloc[i]['tags.name']).add_to(m)
    
# plot map
m